## CLAP implementation from HuggingFace
Zeroshot audio classification

Need to install, datasets, transformers, either tensorflow or pytorch, soundfile, librosa

In [9]:
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio

In [10]:
# details about this particualr dataset available at https://github.com/suzuki256/dog-dataset
dog_barks = load_dataset("437aewuh/dog-dataset", split="train", streaming=True) # dataset of dogbarks. 3 classes: adult, puppy, mix(general dog)

Resolving data files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 1208.60it/s]


In [11]:
dog_barks = dog_barks.shuffle(seed=42, buffer_size=100)  #shuffles the iterable dataset
sample = next(iter(dog_barks))   # selects the first row from the dog_barks 
audio_sample_array = sample["audio"]["array"] # numberical array of the selected sample

In [18]:
# checking the id and verifying the sound
print(sample)
Audio(audio_sample_array, rate = 44100)

{'audio': {'path': 'hf://datasets/437aewuh/dog-dataset@73810687d1afab5e8318f4d7e51bfd7175d1848d/adult_dog/adult_dog_0020.wav', 'array': array([-0.00031829, -0.00022543, -0.00034475, ...,  0.00024021,
        0.00022602,  0.00019103]), 'sampling_rate': 48000}, 'label': 0}


In [13]:
# Different candidate labels for the zero shot evaluation
candidate_labels = ["Sound of a puppy", "Sound of an adult dog","Sound of a dog"]
candidate_labels1 = ["Sound of an toy dog", "Sound of real dog"]
candidate_labels2 = ["Sound of an adult dog", "Sound of a puppy"]

## Loading CLAP model

In [14]:
classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-unfused")

config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.39k/5.39k [00:00<00:00, 3.63MB/s]
pytorch_model.bin: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 615M/615M [00:11<00:00, 55.5MB/s]
tokenizer_config.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 384/384 [00:00<00:00, 577kB/s]
vocab.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 798k/798k [00:00<00:00, 6.86MB/s]
merges.txt: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 456k/456k

In [15]:
classifier(audio_sample_array, candidate_labels=candidate_labels)

[{'score': 0.5428236722946167, 'label': 'Sound of an adult dog'},
 {'score': 0.4466487467288971, 'label': 'Sound of a dog'},
 {'score': 0.010527527891099453, 'label': 'Sound of a puppy'}]

In [16]:
classifier(audio_sample_array, candidate_labels=candidate_labels1)

[{'score': 0.9979725480079651, 'label': 'Sound of real dog'},
 {'score': 0.0020275020506232977, 'label': 'Sound of an toy dog'}]

In [17]:
classifier(audio_sample_array, candidate_labels=candidate_labels2)

[{'score': 0.9809749126434326, 'label': 'Sound of an adult dog'},
 {'score': 0.019025037065148354, 'label': 'Sound of a puppy'}]